# Dropped Reads Analysis Pipeline

Alex Jorjorian 

# Introduction 

The Dropped Reads Analysis Pipeline (DRAP) is an open-source tool I developed to address a critical issue frequently arising in our work at Clear Labs. As a company that develops and markets an end-to-end sequencing assay automation platform, we often utilize short-read sequencers for our assays, including the Sars-Cov2 wastewater surveillance and microbial isolate profiling assays. These assays are each paired with a bioinformatics pipeline that processes the sequencing output to answer biological questions. However, when processing bioinformatics pipelines, remove reads from consideration due to a multitude of problems with the reads; we call these lost reads “dropped reads.” While quality control is essential, a high drop rate of reads belies underlying issues with the quality and cost-effectiveness of our assays.   

In these bioinformatics pipelines, filtering steps are applied so that spurious reads, be they low quality or contaminants, do not confound the results or as a byproduct of mapping reads to a reference genome before further analysis.  Reads are also lost at sample demultiplexing. Losing reads to filtering, demultiplexing, or failed mapping corresponds directly to lost revenue and lost assay power, as every read thrown out represents a spot on a flowcell that is not providing helpful information. A 50% dropped reads rate is a 50% reduction in the number of samples that could have been sequenced to a given quality, all things being equal. A decrease in the number of samples allowed per assay leads to a greater cost per sample, as each run has fixed costs. This increase in cost provides substantial motivation to understand why an assay is losing reads.   

DRAP is designed to be user-friendly, addressing the problem of dropped reads by characterizing user-defined tranches of dropped reads across all samples in a sequencing run. It is a workflow definition language (WDL) pipeline that takes an input zip file and returns an interactive HTML report containing various metrics for each group of dropped reads. This user-friendly format allows scientists without bioinformatics training to easily explore relationships between dropped reads metrics. By studying these reports, scientists can understand the mechanisms by which reads are lost and implement changes to mitigate those losses. For example, if many reads are lost at demultiplexing, there may be an issue with dimerization of amplification primers, which could be filtered out with a more aggressive SPRI treatment. DRAP can also provide rapid customer support by allowing for rapid characterization of low-yield runs. We hope DRAP will be a valuable and accessible tool for institutions or individuals working with short-read sequencing data. 

# Background

Short-read sequencers, such as the various platforms offered by Illumina, generate millions of DNA sequence reads sampled from a solution of DNA fragments known as a library (1). These reads are typically paired such that for each fragment, 150-300bp of the sequence is covered from either end of the fragment with an unsequenced insert in between in the case of fragments longer than the sum of the read lengths. A sequencing assay consists of a series of biochemical manipulations on biological samples such that they produce fragments that can answer a given biological question. For example, a COVID-19 surveillance assay would make a series of fragments from the SARs-Cov2 genome, which would then be sequenced to create a set of reads. A bioinformatics pipeline would then process these reads by mapping the COVID reference, and the variation of consensus alignment could be leveraged to ascertain what strain a sample contained.   

In any bioinformatics pipeline, there are multiple stages where reads may be dropped. We will focus on an archetypal bioinformatics workflow employed at Clear Labs for this proof of concept. The first and often most significant source of dropped reads is demultiplexing, using a sample index on each read to sort the reads into individual samples; this is because to be demultiplexed, the read must rather closely match the index barcode and aberrant reads often do not match (2). Reads that make it into sample fastqs are then usually filtered to meet minimum read length and quality standards using tools such as fastp or trimmomatic. This filtered reads group can be analyzed to understand why they have low-quality scores or short lengths. Finally, reads can be dropped due to failing alignment to some form of reference, be that local alignment like BLAST or global alignment like BWA. This may be due to low quality or the reads coming from an unexpected source.   

DRAP attempts to characterize the primary sources of dropped reads: contamination or off-target amplification, dimerization, low read signal quality, and fragment size. Contamination is a ubiquitous problem in sequencing biological samples, as an utterly sterile environment is impossible outside a clean room. This problem is so widespread that many reference genomes have false variations due to contaminated reads(3). Reads can drop due to contamination and can show up at either demultiplexing if the contaminant DNA fails to ligate to sample index adapters or at an alignment level when they fail to align to the expected reference.   

Dimerization is the process of two sequencing adapters or PCR primers in targeted assays ligating to each other without a sample fragment between them. Many library preparation procedures use size selection to filter out dimers that will inevitably form. However, if even a small amount of dimers enter a PCR reaction, they can outcompete genuine fragments and saturate the flowcell at loading (4, 5). The dimers are dropped at demultiplexing, size filtering, or alignment, depending on pipeline settings.   

Basecall quality score-based read filtering uses Q-scores output by the sequencing instrument to determine if a read is trustworthy enough to pass filtering based on user-defined thresholds. How q-scores are calculated varies by the sequencing platform, considering signal-to-noise ratio, signal intensity, phasing information, and more. Filtering software trims off poor-quality bases and will filter entire reads if their trimmed length or average quality is too low. The underlying causes of low-quality reads are numerous. Some level of quality filtering is inevitable and hard to mitigate as some molecular clusters that become reads simply won’t have enough copies to provide a strong signal. But, some quality failure modalities can be detected by analyzing the reads, the spatial position, and fragment sizes. Bubbles on the flowcell can cause read clusters to miss sequencing cycles, leading to low qscores for those cycles. This would appear as dropped reads spatially colocated in bubble or streak-shaped patterns (1). Overloading can lead to low SNR and show up as low-quality reads next to one another as their light bleeds into one another. These flowcell issues can be ameliorated by altering flowcell loading procedures. Finally, large fragment sizes can lead to low-quality scores as they do not amplify on the flowcell as efficiently and will, therefore, have a lower signal and lower quality; this can be detected by loosely mapping the dropped reads to the target reference and analyzing insert size (6). 

# Design

![Pipeline Diagram](Methods/DRAP.png)
Figure 1: A flow diagram of the various tasks in the DRAP workflow. 

The DRAP is designed to efficiently, scalable, and flexiblely detect and quantify reads dropped by all the mechanisms discussed thus far with maximum portability. To this end, DRAP is written in the Workflow Definition Language (WDL). WDL is an open-source scripting language that several engines can run, most prominently Cromwell and, in the case of this project, miniWDL (7). A WDL workflow comprises multiple task scripts strung together by designating the inputs of one task as the outputs from another. Each task consists of input/output variables, a command section consisting of bash code, and a runtime section defining a docker image that the task will be run in a container based upon. These docker images are webhosted, allowing the WDL tasks to run identically on all systems. WDL is highly scalable as it runs tasks as parallel as possible, given the CPU and memory resources allocated to each task and user hardware. WDL allows anyone to download and run DRAP on their data with little effort. 

The DRAP runs in 3 sequential stages: input processing via consolidate\_samples.wdl, read group processing, and output consolidation. As input, DRAP takes a .json file that defines a list of strings to group fastqs on, the run name and paths to the reference fasta, the Kraken database,  a list of adapters to search for, and a zip file containing all the fastqs and reference fastas to process. The consolidation task then concatenates all fastqs containing the exact group string on a strand basis, yielding an R1 and R2 fastqs file for each group. This is done as the reasons reads are dropped should not vary by sample. The consolidation task also concatenates all fastas in the zip file, and this is then sent to a reference index-building task so that the fasta containing all target sequences can be used in the bwa task. 

After the samples are consolidated, the files for each dropped read group are sent to analysis tasks in parallel. 

![Pipeline Diagram](Methods/Kraken.png)

Figure 2: Example multiqc Kraken Metrics

The Kraken task runs kraken2 with a user-defined database on each group of reads to detect what organisms may contaminate the reads. Kraken2 is a Kmer-based taxonomy calling software that efficiently finds the best fit taxonomic assignment for each read based on a reference database, in our case, a reasonably extensive refseq PlusPF-16 database (8). If a significant portion of reads dropped at any step are from an unexpected organism, scientist can rework their assay to mitigate the risk of contamination. 

![Pipeline Diagram](Methods/Fastqc.png)

Figure 3: Example multiqc fastqc metrics

The fastqc task runs the fastq quality analysis tool separately on each R1 and R2 fastq file. This provides many read quality metrics, from quality score by cycle to duplicate sequence counts. By aggregating the fastqc metrics for each dropped reads group, DRAP gives end users a high degree of flexibility to investigate many questions about why read quality is poor. 

![Pipeline Diagram](Methods/Flowcell.png)

Figure 4: Example of flowcell metrics plots and tables

The flowcell metrics task consists of custom-written Python code that leverages the read id string for each read in the fastq files to analyze the positions of each dropped read. These positions are processed to produce a plot of the positions of each read that can be used to visually inspect if there are any spatial patterns to dropped read positions indicative of bubbles or flowcell defects. These positions are also processed with the clustering algorithm Scikit Learn dbscan to detect and output any clustered read positions, which may indicate bubbles or aggregates (9). The clusters are also plotted as an image, and a metric file tracking the cluster sizes and counts is returned. An accessory task in the results consolidation stage processes the outputs from each instantiation of flowcell metrics into a format compatible with multiqc.  

![Pipeline Diagram](Methods/Dimers.png)

Figure 5: Example of dimer metrics output 

The dimer metrics task runs bbduk from the bbmap suite of bioinformatic tools and processes the outputs into a convenient format. BBduk is a tool used to detect and trim adapter sequences from input reads, but for the DRAP, it is used to detect the percentage of input reads that contain adapter or primer sequences defined by the user (10). This provides one method for the user to estimate the prevalence of primer or adapter dimers in each dropped read group. This does require the user to know the primer and adapter sequences used in the assay. The results for all read groups are combined into a multiqc compatible file in a separate consolidation task. 

![Pipeline Diagram](Methods/Mapping.png)

Figure 6: Examples of loose mapping metrics

For alignment analysis DRAP employs a subworkflow, bwa loose mapping metrics. As the name implies, the task runs the BWA mem alignment algorithm (11)  with highly permissive settings to maximize mapping, allowing DRAP to capture as much information as possible about potentially low-quality dropped reads. BWA is an industry-standard global alignment algorithm that outputs sam files of read alignments. The loose mapping workflow then passes these alignments to the bamStat task that processes the alignments with samtools stats (12) to provide general mapping metrics and Picard tools collectInserSizeMetrics to calculate the insert size distributions for each read group (13). The mapping metrics allow the user to understand how many reads in each dropped read group are mapped to one of the target reference sequences, giving a secondary indication of contamination. Samtools also returns a series of other secondary metrics that could indicate the quality of the reads and their composition. As discussed earlier, the insert size metrics allow the user to inspect the insert size distribution for excessively long inserts that may lead to lower read quality or very short fragment sizes indicative of dimerization. 

![Pipeline Diagram](Methods/Stats.png)
Figure 7: Example of the multiqc summary table

Once the analysis and analysis-specific consolidation steps are run, their output files are passed on to the multiqc task. This task contains a simple bash script that runs multiqc on all the output files, consolidating them into a single interactive html report. Multiqc is a powerful bioinformatics report aggregation tool that can search a directory for output files from over 140 different bioinformatics tools and consolidate them into a single portable HTML report (14). Multiqc also allows users to leverage its custom file processing utilities to add custom sections to the report using specially structured file formats. DRAP does this for the flowcell metrics and dimer metrics reports. Multiqc QC reports also allow users to create new visualizations, such as scatterplots, extending their utility from the data they contain. The final output of each DRAP run is a single html file using the user-defined report name as the title. 

The DRAP code's fine details are beyond this report's scope, but all files are publicly available in the linked GitHub repository (add link). All tasks in the DRAP use custom-built Docker images built off open-source base images; these are uploaded to Docker Hub and are publicly available. The DRAP repo is broken up into subdirectories: Docker containing the Dockerfiles for each image along with code to build them; a resources directory containing default references and accessory files; a task directory containing each WDL task file; and a workflow directory containing the DRAP workflow. An additional Google Drive repository (linked here) includes the kraken2 database used in this project and all test data zip files. 

## Materials and Testing Methodology 

The DRAP pipeline was tested on data from 12 Clear Labs runs from two short read assays, microbial surveillance, and SARS-Cov2 wastewater. These runs were selected from runs our field service team deemed problematic, development runs we had known problems with, and runs that had dropped reads metrics, filtering rate, mapping coverage, and mapping rate far below expectations based on a database query. To extract the dropped reads, our existing bioinformatic pipelines were modified to save reads dropped at demux, each filtering step, and alignment steps to a zip file, along with any references used for alignment. The code changes to extract this data are outside this project's scope. All output reports and an index describing the run will be available on Google Drive. (add link) In the results section, I will describe a subset of these runs and provide examples of how the DRAP HTML reports provide utility in troubleshooting why reads are dropped in bioinformatics analysis of sequencing assays. 

# Results  

The DRAP was run on the 12 datasets outlined above during development and testing. In this section, I will first discuss the DRAP's computational performance and then highlight the 3 case studies from the testing set. 

## Runtime Analysis

![Runtime Analysis](Runtime/Runtimes.png)

Figure 8: Regression plots of total runtime and total CPU seconds per MB of input data

![Runtime Analysis](Runtime/Tasks.png)

Figure 9: Boxplots of task runtime in CPU seconds. 

The DRAP must be maximally performant across various hardware platforms to maximize its utility and ease of adoption. To investigate this, I have plotted the total clock time and CPU time utilized by the DRAP across the 12 test runs against the total data processed in Mb. All test runs were run locally on a 16-thread 32GB memory Linux machine. As shown in Figure 8, the CPU runtime is only 12.53 seconds per MB, while the total runtime is slower at 15.7 seconds per MB. This indicates bottlenecks in the pipeline on this hardware where tasks are idle while waiting for resources. This is likely in the Kraken task as it is allocated 32 GB of memory, meaning only one instance can be run simultaneously. We also see outlier runs that take far longer than average for a given dataset size, meaning some tasks must be hitting a slowdown. Figure 9 shows that the task with the highest CPU runtimes in BWA alignment, with some outlier instances taking several CPU hours to process, is likely due to many mappable reads in one of the read groups. The Flowcell Metrics task is typically quite fast; however, there are instances when it takes over a CPU hour. This is due to the dbscan clustering algorithm finding many candidate clusters in some runs. Future versions of DRAP may seek to implement optimizations to increase clustering speed. The generation code for this runtime analysis is available in the appendix and on Git Hub. 

## Case Study 1 

![Multiqc Summary Table](Case1/Table.png)

Figure 10: Multiqc Summary Table from case study 1

![Base Composition](Case1/R1_bases.png)

Figure 11: Read 1 bases are evenly balanced

![Base Composition](Case1/R2_Bases.png)

Figure 12: Read 2 bases are skewed towards AT

![Quality Scores](Case1/QualityShift.png)

Figure 13: Quality scores are lower in the second read of the undermined group

Case study 1 involves a development run from our Microbial Isolate assay that we determined was contaminated with polyA sequences. In this run, all samples were listeria isolates. In the DRAP report, it is immediately apparent that there is a high degree of GC bias in the dropped read groups, see Figure 10. Investigating further in the fastq report section, the first strand is somewhat balanced in the base composition per cycle (Figure 11). Still, the second strand (Figure 12) is highly enriched in thymine basecalls, consistent with a poly A sequence ligated to the end of many fragments in the run. We also see a stark drop in average q-score in the second strand of undermined reads, and this is consistent with many in-phase poly A  clusters reducing the SNR and, consequently, the quality score of nearby clusters (Figure 13). If DRAP reports were available during development, this failure modality would have been detected more quickly, allow for more rapid troubleshooting and shorter development time. 

## Case Study 2 

![Kraken Phylum Breakdown](Case2/Kraken.png)

Figure 14: Phylum level taxonomic breakdown from case study 2. 


For the second case study, I selected a run that the customer flagged as contaminated to see if DRAP could allow for easier contamination detection. Upon inspecting the Kraken outputs of the DRAP, seen in figure n we see that 4 million read pairs from the unmapped reads group are assigned to various bacterial phylum.  The run is from our microbial isolate assay, in which each sample is derived from a microbial monoculture, a fact that is leveraged to select the reference genome to align to for each sample. For this run the reference genomes used were from species in the Pseudomonadota and Bacilliota phyla, however we also see Bacteroidota in the Kraken phylum breakdown. Given that the run is based on isolates the only mechanism to find 10% of reads from a different phylum is contamination. When this run was escalated to our customer support team, a bioinformatics scientist was pulled from other tasks to confirm the failure mode. With the DRAP, the customer support team could simply look at the HTML report and tell at a glance that contamination was present. If contamination exists it should typically show up in unmapped reads as they will fail to align to the expected reference.


## Case Study 3

For the final case study, I utilized the Clear Labs big query database to find a production Wastewater surveillance run with poor success metrics, low coverage of the Sars-Cov2 reference genome, low read mapping rate, and a high degree of filtering. This run is typical of the type of run a customer might escalate to our support team, making it a perfect use case for using DRAP reports to ascertain the root cause of poor performance.  





# Discussion and Future Efforts

As the case studies described above show, the Dropped Reads Analysis Pipeline (DRAP) could be a force multiplier in developing and troubleshooting short-read sequencing assays. The DRAP is a relatively lightweight pipeline that can be quickly integrated into existing WDL-based workflows and other workflows with a small amount of wrapper code. Of particular utility is the Kraken-based contamination module, which is highly useful in troubleshooting unknown issues. In this project, I have found a few shortcomings of the pipeline; for example, the flowcell metrics module has yet to show any utility and requires optimization to cluster read positions efficiently. It is still of interest to leave in the pipeline however, as I was unable to source test runs that contained flowcell or bubbles, this does not mean that once implemented, these will not be detected.  The dimer metrics were also limited in my case studies by the list of sequences supplied to it to search for dimers upon, while in the undermined read groups, it consistently found thousands of Illumina adapter dimers. Case study one shows that it needed to be adequately designed to detect primer dimers from amplicon tiling-based assays. Future development will overhaul this module to detect amplicon primer dimers without marking all read from the amplicons as dimers, as this is the reason I did not supply the task with the ArticV5 primer set used in the Wastewater surveillance product. Additional development efforts will seek to extend the DRAP to long-read sequencing assays, add additional tasks to profile reads dropped by the basecalling software, and general runtime optimizations. Despite these challenges, the DRAP is ready to be integrated into Clear Labs development pipelines and utilized in an Open-source manner. 

# Appendix 

### Source code:

All DRAP source code is located at the attached github repo,   
DRAP is open source and free to use with citation. 

### Test Run Data:

All test run data is located in the attached public Google Drive link.   
The case studies are in zip files named case Studies 1, 2, and 3, respectively.   
The following code block will download all the test runs and a directory containing their respective multiqc reports into a folder named project data in your working directory. 

### Drap Pipeline Installation and Testing 

The code block below will attempt to install the DRAP dependencies onto your active environment. There are many mechanisms by which this may fail. Therefore, I will also outline the steps to install the dependencies.  

1. First make sure you have python 3.7 or above  
1. Next install docker as described [here](https://www.docker.com/get-started/)   
1. Install miniwdl and docker filing using the procedure outlined [here](https://github.com/chanzuckerberg/miniwdl?tab=readme-ov-file\#install-miniwdl).   
1. Next clone the DRAP GitHub repo and checkout the branch current

The code block below should run DRAP on the 3 case study datasets, however should that fail one can run DRAP with the command outline below. 

miniwdl run path\_to\_drap\_repo/workflows/DroppedReadsAnalysis.wdl \-i path\_to\_project\_data/CaseN.json 

# References 

1. Ravi, R.K., Walton, K., Khosroheidari, M. (2018). MiSeq: A Next Generation Sequencing Platform for Genomic Analysis. In: DiStefano, J. (eds) Disease Gene Identification. Methods in Molecular Biology, vol 1706\. Humana Press, New York, NY. https://doi.org/10.1007/978-1-4939-7471-9\_12   
1. Hu, T., Chitnis, N., Monos, D., & Dinh, A. (2021). Next-generation sequencing technologies: An overview. Human Immunology, 82(11), 801-811. [https://doi.org/10.1016/j.humimm.2021.02.012](https://doi.org/10.1016/j.humimm.2021.02.012)  
1. Goig, G.A., Blanco, S., Garcia-Basteiro, A.L. et al. Contaminant DNA in bacterial sequencing experiments is a major source of false genetic variability. BMC Biol 18, 24 (2020). [https://doi.org/10.1186/s12915-020-0748-z](https://doi.org/10.1186/s12915-020-0748-z)  
1. Shore S, Henderson JM, Lebedev A, Salcedo MP, Zon G, McCaffrey AP, et al. (2016) Small RNA Library Preparation Method for Next-Generation Sequencing Using Chemical Modifications to Prevent Adapter Dimer Formation. PLoS ONE 11(11): e0167009. [https://doi.org/10.1371/journal.pone.0167009](https://doi.org/10.1371/journal.pone.0167009)  
1. Garafutdinov, R. R., Galimova, A. A., & Sakhabutdinova, A. R. (2020). The influence of quality of primers on the formation of primer dimers in PCR. Nucleotides, Nucleotides & Nucleic Acids, 39(9), 1251–1269. [https://doi.org/10.1080/15257770.2020.1803354](https://doi.org/10.1080/15257770.2020.1803354)  
1. Tan, G., Opitz, L., Schlapbach, R., & Rehrauer, H. (2019). Long fragments achieve lower base quality in Illumina paired-end sequencing. Scientific Reports, 9(1), 1-7. [https://doi.org/10.1038/s41598-019-39076-7](https://doi.org/10.1038/s41598-019-39076-7)  
1. Voss K, Van der Auwera G and Gentry J. Full-stack genomics pipelining with GATK4 \+ WDL \+ Cromwell \[version 1; not peer reviewed\]. F1000Research 2017, 6(ISCB Comm J):1381 (slides) DOI:10.7490/f1000research.1114634.1  
1. Wood, D.E., Lu, J. & Langmead, B. Improved metagenomic analysis with Kraken 2\. *Genome Biol* 20, 257 (2019). https://doi.org/10.1186/s13059-019-1891-0  
1. Scikit-learn: Machine Learning in Python, Fabian Pedregosa, Gaël Varoquaux, Alexandre Gramfort, Vincent Michel, Bertrand Thirion, Olivier Grisel, Mathieu Blondel, Peter Prettenhofer, Ron Weiss, Vincent Dubourg, Jake Vanderplas, Alexandre Passos, David Cournapeau, Matthieu Brucher, Matthieu Perrot, Édouard Duchesnay; 12(85):2825−2830, 2011\.  
1. Bushnell, Brian. BBMap: A Fast, Accurate, Splice-Aware Aligner. United States: N. p., 2014\. Web.  
1. Li, H. "Aligning sequence reads, clone sequences and assembly contigs with BWA‐MEM." arXiv (2013).  
1. Heng Li, Bob Handsaker, Alec Wysoker, Tim Fennell, Jue Ruan, Nils Homer, Gabor Marth, Goncalo Abecasis, Richard Durbin, 1000 Genome Project Data Processing Subgroup, The Sequence Alignment/Map format and SAMtools, Bioinformatics, Volume 25, Issue 16, August 2009, Pages 2078–2079, [https://doi.org/10.1093/bioinformatics/btp352](https://doi.org/10.1093/bioinformatics/btp352)  
1. “Picard Toolkit.” 2019\. Broad Institute, GitHub Repository. https://broadinstitute.github.io/picard/; Broad Institute  
1. Philip Ewels, Måns Magnusson, Sverker Lundin, Max Käller, MultiQC: summarize analysis results for multiple tools and samples in a single report, Bioinformatics, Volume 32, Issue 19, October 2016, Pages 3047–3048, https://doi.org/10.1093/bioinformatics/btw354